In [ ]:
import pandas as pd
import os
import requests
import urllib.parse
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from mtcnn.mtcnn import MTCNN
from io import BytesIO
import hashlib
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam

# Instantiate the MTCNN detector
detector = MTCNN()

def download_image(url, dirname="images"):
    filename = os.path.join(dirname, hashlib.md5(url.encode()).hexdigest() + ".jpg")
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)
    return filename

def process_image(filename):
    try:
        image = Image.open(filename)
    except IOError:
        print(f"Cannot open {filename}. It might not be a valid image file.")
        return None
    image = image.convert('RGB')
    pixels = np.asarray(image)

    # Detect faces in the image
    results = detector.detect_faces(pixels)
    
    # Skip images if no face is detected
    if len(results) == 0:
        print(f"No face detected in {filename}")
        return None

    # Extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height

    # Extract the face
    face = pixels[y1:y2, x1:x2]

    # Resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize((224, 224))
    return np.array(image)

# Read your dataframe
df = pd.read_excel("output.xlsx")

if not os.path.exists("images"):
    os.makedirs("images")

# Download and process images
df['image'] = df['image_link'].apply(lambda url: process_image(download_image(url)))

X = np.stack(df['image'])
y = df['label'].values

# Encode the labels into numbers
le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # We add dense layers so that the model can learn more complex functions and classify for better results.
output = Dense(len(le.classes_), activation='softmax')(x)  # Final layer with softmax activation for N classes

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Train the model
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          steps_per_epoch=len(X_train) / 32, 
          validation_data=(X_test, y_test),
          epochs=10)
